##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# "Επικάλυψη"(masking) και "γέμισμα"(padding) με την Keras

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/keras/masking_and_padding">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    Άνοιγμα στο TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/el/guide/keras/masking_and_padding.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Εκτέλεση στο Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/el/guide/keras/masking_and_padding.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    Προβολή πηγαίου στο GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/el/guide/keras/masking_and_padding.ipynb">
    <img src="https://www.tensorflow.org/images/download_logo_32px.png" />
    Λήψη "σημειωματάριου"</a>
  </td>
</table>

Note: Η κοινότητα του TensorFlow έχει μεταφράσει αυτά τα έγγραφα. Καθότι οι μεταφράσεις αυτές αποτελούν την καλύτερη δυνατή προσπάθεια , δεν υπάρχει εγγύηση ότι θα παραμείνουν ενημερωμένες σε σχέση με τα [επίσημα Αγγλικά έγγραφα](https://www.tensorflow.org/?hl=en).
Αν έχετε υποδείξεις για βελτίωση των αρχείων αυτών , δημιουργήστε ένα pull request στο [tensorflow/docs](https://github.com/tensorflow/docs) GitHub repository . Για να συμμετέχετε στη σύνταξη ή στην αναθεώρηση των μεταφράσεων της κοινότητας , επικοινωνήστε με το [docs@tensorflow.org list](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs).

# Διάταξη


In [ ]:
import numpy as np

import tensorflow as tf

from tensorflow.keras import layers

# "Γέμισμα" σειριακών δεδομένων

Κατά την επεξεργασία ακολουθιών δεδομένων, είναι σύνηθες ορισμένα δείγματα να έχουν διαφορετικό μήκος. Για παράδειγμα (το κείμενο ομαδοποιείται κατά λέξεις) :

```
[
  ["The", "weather", "will", "be", "nice", "tomorrow"],
  ["How", "are", "you", "doing", "today"],
  ["Hello", "world", "!"]
]
```
Μετά από λεξιλογική αναζήτηση , τα δεδομένα μπορούν να εμφανιστούν ως ακέραιοι :

```
[
  [83, 91, 1, 645, 1253, 927],
  [73, 8, 3215, 55, 927],
  [71, 1331, 4231]
]
```

Το παραπάνω πρόκειται για μία δισδιάστατη λίστα όπου ορισμένα δείγματα έχουν μήκος 6,5, και 3 αντίστοιχα. Επειδή τα δεδομένα εισόδου για ένα μοντέλο μάθησης εις βάθος πρέπει να είναι ένας τανυστής(π.χ `batch_size, 6 , vocab_size` στην παραπάνω περίπτωση), δείγματα τα οποία είναι μικρότερα από το μεγαλύτερο στοιχείο , πρέπει να "γεμίσουν" με κάποια συμβολική τιμή υποκατάστασης (εναλλακτικά, κάποιος μπορεί να "κόψει" τα μεγάλα δείγματα αντί να "γεμίσει" τα μικρότερα σε μήκος δείγματα).

Η Keras παρέχει ένα API μέσω του οποίου μπορείτε εύκολα να "κόψετε" και να "γεμίσετε" τις ακολουθίες δεδομένων : `tf.keras.preprocessing.sequence.pad_sequences`

In [ ]:
raw_inputs = [
  [83, 91, 1, 645, 1253, 927],
  [73, 8, 3215, 55, 927],
  [711, 632, 71]
]

# By default, this will pad using 0s; it is configurable via the
# "value" parameter.
# Note that you could "pre" padding (at the beginning) or
# "post" padding (at the end).
# We recommend using "post" padding when working with RNN layers
# (in order to be able to use the 
# CuDNN implementation of the layers).
padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(raw_inputs,
                                                              padding='post')

print(padded_inputs)

# "Επικάλυψη"

Τώρα που όλα τα δείγματα έχουν όμοιο μήκος , το μοντέλο πρέπει κάπως να ενημερωθεί ότι μέρος των δεδομένων είναι στην πραγματικότητα "γέμισμα" και πρέπει να αγνοηθεί. Ο μηχανισμός αυτός λέγεται "επικάλυψη".

Υπάρχουν τρείς τρόποι εισαγωγής "επικάλυψης" στα μοντέλα Keras :

* Προσθήκη ενός επιπέδου `keras.layers.Masking`.
* Ρύθμιση ενός επιπέδου `keras.layers.Embedding` με `mask_zero=True.
* Πέρασμα ενός ορίσματος `mask` χειροκίνητα , όταν καλούνται επίπεδα που το υποστηρίζουν (π.χ RNN επίπεδα).

# Επίπεδα "επικάλυψης" : ενσωμάτωση και "επικάλυψη"

Στο παρασκήνιο, αυτά τα επίπεδα θα δημιουργήσουν έναν τανιστή επικάλυψης(2D τανιστής με μορφή `(batch, sequence_length())`,και θα τον ενσωματώσουν στον τανιστή εξόδου,τον οποίο επιστρέφει το επίπεδο `Επικάλυψης` ή `Ενσωμάτωσης`.


In [ ]:
embedding = layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True)
masked_output = embedding(padded_inputs)

print(masked_output._keras_mask)

In [ ]:
masking_layer = layers.Masking()
# Simulate the embedding lookup by expanding the 2D input to 3D,
# with embedding dimension of 10.
unmasked_embedding = tf.cast(
    tf.tile(tf.expand_dims(padded_inputs, axis=-1), [1, 1, 10]),
    tf.float32)

masked_embedding = masking_layer(unmasked_embedding)
print(masked_embedding._keras_mask)

Η μάσκα είναι ένας δισδιάστατος boolean τανιστής με μορφή `(batch_size , sequence_length)`,όπου κάθε `Ψευδής` καταχώριση υποδεικνύει ότι το αντίστοιχο χρονικό σημείο θα πρέπει να αγνοηθεί κατά την επεξεργασία. 

# Αναπαραγωγή επικάλυψης στα Functional API και Sequential API

Όταν χρησιμοποιείται το Functional ή το Sequential API , μία επικάλυψη που έχει δημιουργειθεί από ένα επίπεδο `Ενσωμάτωσης` ή `Επικάλυψης` θα αναπαραχθεί εντός του δικτύου για κάθε επίπεδο το οποίο είναι σε θέση να τη χρησιμοποιήσει(π.χ RNN επίπεδα). Η επικάλυψη που αντιστοιχεί σε μία είσοδο περνά σε κάθε επίπεδο που μπορεί να τη χρησιμοποιήσει - αυτό γίνεται αυτόματα από την Keras -.

Σημειώνεται ότι στη μέθοδο `call` μίας υποκλάσσης μοντέλου ή επιπέδου , οι μάσκες δεν αναπαράγωνται αυτόματα , οπότε θα χρειαστεί να περάσει χειροκίνητα το όρισμα `mask` σε όποιο επίπεδο το χρειάζεται. Δείτε παρακάτω για λεπτομέρειες.

Π.χ , στο ακόλουθο Sequential μοντέλο , το επίπεδο `LSTM` θα δεχτεί αυτόματα τη μάσκα , οπότε θα αγνοήσει τις "γεμισμένες" τιμές :

In [ ]:
model = tf.keras.Sequential([
  layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True),
  layers.LSTM(32),
])

Όμοια ,  για το Functional API μοντέλο : 

In [ ]:
inputs = tf.keras.Input(shape=(None,), dtype='int32')
x = layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True)(inputs)
outputs = layers.LSTM(32)(x)

model = tf.keras.Model(inputs, outputs)

# Πέρασμα τανιστών "επικάλυψης" απευθείας σε επίπεδο

Επίπεδα τα οποία μπορούν να διαχειριστούν μάσκες(όπως το `LSTM`) έχουν όρισμα `mask` στην μέθοδο `__call__` τους.

Στο μεταξύ , επίπεδα τα οποία παράγουν μάσκες (π.χ `Ενσωμάτωσης`) εκθέτουν την μέθοδο `compute_mask(input, previous_mask)` την οποία μπορείτε να καλέσετε.

Επομένως, μπορείτε να κάνετε κάτι τέτοιο :


In [ ]:
class MyLayer(layers.Layer):
  
  def __init__(self, **kwargs):
    super(MyLayer, self).__init__(**kwargs)
    self.embedding = layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True)
    self.lstm = layers.LSTM(32)
    
  def call(self, inputs):
    x = self.embedding(inputs)
    # Note that you could also prepare a `mask` tensor manually.
    # It only needs to be a boolean tensor
    # with the right shape, i.e. (batch_size, timesteps).
    mask = self.embedding.compute_mask(inputs)
    output = self.lstm(x, mask=mask)  # The layer will ignore the masked values
    return output

layer = MyLayer()
x = np.random.random((32, 10)) * 100
x = x.astype('int32')
layer(x)

# Υποστήριξη masking σε δικά σας layers 

Μερικές φορές , μπορεί να χρειαστεί να γράψετε επίπεδα τα οποία παράγουν μία μάσκα (π.χ `Ενσωμάτωσης`) ή επίπεδα τα οποία θα χρειάζεται να τροποποιούν την τρέχουσα μάσκα.

Για παράδειγμα, κάθε επίπεδο το οποίο παράγει έναν τανιστή με διαφορετική διάσταση χρόνου από ότι στην είσοδο του , όπως το επίπεδο `Συγχώνευση` (Concatenate) το οποίο συγχωνεύει στη διάσταση του χρόνου, θα χρειάζεται να τροποποιεί την τρέχουσα μάσκα έτσι ώστε τα χαμηλότερα επίπεδα να μπορούν να λάβουν υπ'όψιν τις κατάλληλες χρονοσημάνσεις.

Για να γίνει το παραπάνω,το επίπεδο σας πρέπει να υλοποιεί την μέθοδο `layer.compute_mask()`, η οποία παράγει μία νέα μάσκα , με βάση  τη δοσμένη είσοδο και την τρέχουσα μάσκα

Τα περισσοτερα επίπεδα δεν τροποποιούν τη διάσταση του χρόνου , επομένως δεν χρειάζεται να ανησυχείτε για την επικάλυψη. Η προεπιλεγμένη συμπεριφορά της `compute_mask()` είναι απλά να περνάει την τρέχουσα μάσκα στο δίκτυο , όταν αυτό χρειάζεται.
Ακολουθεί παράδειγμα με το επίπεδο `TemporalSplit` το οποίο χρειάζεται να τροποποιήσει την τρέχουσα μάσκα: 

In [ ]:
class TemporalSplit(tf.keras.layers.Layer):
  """Split the input tensor into 2 tensors along the time dimension."""

  def call(self, inputs):
    # Expect the input to be 3D and mask to be 2D, split the input tensor into 2
    # subtensors along the time axis (axis 1).
    return tf.split(inputs, 2, axis=1)
    
  def compute_mask(self, inputs, mask=None):
    # Also split the mask into 2 if it presents.
    if mask is None:
      return None
    return tf.split(mask, 2, axis=1)

first_half, second_half = TemporalSplit()(masked_embedding)
print(first_half._keras_mask)
print(second_half._keras_mask)

Άλλο ένα παράδειγμα , ενός επιπέδου `CustomEmbedding` το οποίο είναι ικανό να παράγει μάσκα από τις δοσμένες τιμές εισόδου :

In [ ]:
class CustomEmbedding(tf.keras.layers.Layer):
  
  def __init__(self, input_dim, output_dim, mask_zero=False, **kwargs):
    super(CustomEmbedding, self).__init__(**kwargs)
    self.input_dim = input_dim
    self.output_dim = output_dim
    self.mask_zero = mask_zero
    
  def build(self, input_shape):
    self.embeddings = self.add_weight(
      shape=(self.input_dim, self.output_dim),
      initializer='random_normal',
      dtype='float32')
    
  def call(self, inputs):
    return tf.nn.embedding_lookup(self.embeddings, inputs)
  
  def compute_mask(self, inputs, mask=None):
    if not self.mask_zero:
      return None
    return tf.not_equal(inputs, 0)
  
  
layer = CustomEmbedding(10, 32, mask_zero=True)
x = np.random.random((3, 10)) * 9
x = x.astype('int32')

y = layer(x)
mask = layer.compute_mask(x)

print(mask)

# Συγγραφή επιπέδων που χρειάζονται πληροφορίες της μάσκας

Μερικά επίπεδα είναι *καταναλωτές*  μασκών · δέχονται ένα όρισμα `mask` στην `call` και το χρησιμοποιούν ώστε να προσδιορίσουν αν θα αγνοήσουν συγκεκριμένες χρονοσημάνσεις.
Για να γράψετε είναι τέτοιο επίπεδο , μπορείτε απλά να προσθέσετε το όρισμα `mask=None` στην υπογραφή της μεθόδου `call`. Η μάσκα που συσχετίζεται με τις εισόδους θα περάσει στο επίπεδο σας όποτε είναι διαθέσιμη.

```python
class MaskConsumer(tf.keras.layers.Layer):
  
  def call(self, inputs, mask=None):
    ...
```


# Σύνοψη

Αυτά αποτελούν όσα χρειάζεστε για το masking στην Keras . Για να συνοψίσουμε :

* "Masking" είναι ο τρόπος ,με τον οποίο τα επίπεδα είναι ικανά να γνωρίζουν πότε να αγνοήσουν συγκεκριμένες χρονοσημάνσεις σε ακολουθίες δεδομένων.
* Μερικά επίπεδα είναι γεννήτορες μασκών : το `Ενσωμάτωση` μπορεί να παράγει μάσκα από τιμές εισόδου (αν `mask_zero=True`), ομοιώς και το επίπεδο `Masking`.
* Μερικά επίπεδα είναι καταναλωτές μασκών : "εκθέτουν" ένα όρισμα `mask` στην μέθοδο `__call__` τους . Αυτή είναι η περίπτωση των RNN δικτύων.
* Στα Functional και Sequential API's , οι πληροφορίες επικάλυψης διαχέονται αυτόματα.
* Κατά την σύνταξη υποκλάσσεων μοντέλων ή όταν χρησιμοποιούνται επίπεδα κατά αυτοτελή τρόπο , το όρισμα `mask` πρέπει να περνάει στα επίπεδα χειροκίνητα.
* Μπορείτε εύκολα να συντάξετε επίπεδα τα οποία να τροποποιούν την τρέχουσα μάσκα, να παράγουν μία νέα μάσκα ,ή να καταναλώνουν την μάσκα που συσχετίζεται με την είσοδο. 